# Contador de Dedos

![land_mark](hand_landmarks.png)

In [46]:
## Importacion de librerias
import cv2 as cv
import mediapipe as mp
import time

## referenciando metodos necesarios
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# configurando parametros de deteccion
hands = mp_hands.Hands(max_num_hands=1, # numero de manos a detectar
                       min_detection_confidence=0.8,  # porcentaje de precisión en la detección
                       min_tracking_confidence=0.8  # porcentae de precision en el tracking
                      )
# creacion de ventanas para mostrar los resultados
cv.namedWindow("MediaPipe Hands", cv.WINDOW_NORMAL)
time_ant=0
## *****************************************************************************
## Funcion para dibujar los puntos y sus conexiones
def hand_detect(img):
    global time_ant
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB) # cambiamos el espacio de color de la imagen porque mediapipe 
                                            # acepta el eapcio RGB
    
    result = hands.process(img) # realizamos la inferencia en la imagen usando el metodo process
    num_dedos = 0 #variable para contar el numero de dedos
    h,w,c = img.shape # extraemos las dimensiones de la imagen 
    # si se ha detectado una mano
    if result.multi_hand_landmarks: 
        # iteramos entre las manos detectadas
        for hand_landmarks in result.multi_hand_landmarks:
            #dibujamos las conexiones de los puntos clave de la mano
            mp_drawing.draw_landmarks(img, 
                                      hand_landmarks, 
                                      mp_hands.HAND_CONNECTIONS,
                                      mp_drawing_styles.get_default_hand_landmarks_style(),
                                      mp_drawing_styles.get_default_hand_connections_style()
                                      )
            hlm = [] # lista para almacenar las coordenadas de cadapunto de la mano 
            # iteramos entre los puntos de cada mano 
            for id, lm in enumerate(hand_landmarks.landmark):
                # almacenamos los puntos de cada mano 
                hlm.append([id, int(lm.x*w), int(lm.y*h)])
                
            # almacenamos en una lista nuestros puntos de interes
            dedos = [8, 12, 16, 20]
            # iteramos en nuestros puntos de interes 
            for id, lm in enumerate(dedos):
                # nuestros puntos de Interes expresan los puntos que expresan las puntas de los dedos
                # la comparacion dice que si la dimension en y del punto de la punta del dedo es menor
                # que la dimension en y del punto de la base del dedo, el dedo esta cerrado
                # caso contrario el dedo esta abierto
                if hlm[lm][2] < hlm[lm-2][2]:
                    num_dedos += 1
                    
            dedo_gordo = abs(hlm[4][1]-hlm[5][1])
            #print(dedo_gordo)
            if (dedo_gordo > 50):
                num_dedos += 1
    # escribimos la cantidad de deos detectados                
    cv.putText(img, 
               "Dedos:"+str(num_dedos), 
                (w//2-w//6,40), 
                cv.FONT_HERSHEY_SIMPLEX, 
                1.5, 
                (255,0,0),2)
    
    #Calculamos el numerod e frames por segundo
    time_pos = time.time()
    fps = 1 / (time_pos - time_ant)
    time_ant = time_pos
    # Escribimos el numro de frames por segundo
    cv.putText(img, 
               "fps:"+str(fps), 
                (w-100,40), 
                cv.FONT_HERSHEY_SIMPLEX, 
                1, 
                (255,0,0),2)
            
                
    # realizamos el cambio deespaciod e color            
    img = cv.cvtColor(img, cv.COLOR_RGB2BGR)
    # retonamos la imagen graficada
    return img
## *****************************************************************************
## Funcion para dibujar los cuadros delimitadores

               
## *****************************************************************************
## ----------- CODIGO PRINCIPAL--------------------------
## creacion del Objeto captura de video 
cap = cv.VideoCapture(0)
wCam, hCam = 720, 560
cap.set(3, wCam)
cap.set(4, hCam)


while cap.isOpened():
    
    ret, frame = cap.read() 
    
    if ret:
        frame = cv.flip(frame, 1)
        frame[:50,:] = (200,200,0)
        img_proces = frame.copy()
        image = hand_detect(img_proces)
        cv.imshow("MediaPipe Hands", image)
        
        key = cv.waitKey(10)
        if key == 27:
            break
        
    else:
        break
            
cap.release()
cv.destroyAllWindows()